In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(dim))
        self.register_buffer('beta', torch.zeros(dim))

    def forward(self, x):
        return F.layer_norm(x, x.shape[-1:], self.gamma, self.beta)

In [ ]:
# 함수 형태로
def Normalize(in_channels, norm_type='group', num_groups=32):
    if norm_type == 'batchnorm':
        return torch.nn.BatchNorm2d(num_features=in_channels)
    else:
        return torch.nn.GroupNorm(num_groups=num_groups, num_channels=in_channels, eps=1e-6, affine=True)

In [ ]:
# affine transform 없는 버전(학습 안하는 버전)
from typing import Union, List

import torch


batch_size, seq_size, embed_dim = 2, 3, 4
embedding = torch.randn(batch_size, seq_size, embed_dim)
print("x: ", embedding)
print(embedding.shape)
print()


layer_norm = torch.nn.LayerNorm(embed_dim, elementwise_affine=False)
layer_norm_output = layer_norm(embedding)
print("y: ", layer_norm_output)
print(layer_norm_output.shape)
print()


def custom_layer_norm(
        x: torch.Tensor, dim: Union[int, List[int]] = -1, eps: float = 0.00001
) -> torch.Tensor:
    mean = torch.mean(x, dim=(dim,), keepdim=True)
    var = torch.square(x - mean).mean(dim=(dim,), keepdim=True)
    return (x - mean) / torch.sqrt(var + eps)


custom_layer_norm_output = custom_layer_norm(embedding)
print("y_custom: ", custom_layer_norm_output)
print(custom_layer_norm_output.shape)

assert torch.allclose(layer_norm_output, custom_layer_norm_output), 'Tensors do not match.'

In [ ]:
class Network(torch.nn.Module):
    """
    LayerNorm 초기화 편하게 하기
    """
    @staticmethod
    def calc_activation_shape(
        dim, ksize, dilation=(1, 1), stride=(1, 1), padding=(0, 0)
    ):
        def shape_each_dim(i):
            odim_i = dim[i] + 2 * padding[i] - dilation[i] * (ksize[i] - 1) - 1
            return (odim_i / stride[i]) + 1


        return shape_each_dim(0), shape_each_dim(1)


    def __init__(self, idim, num_classes=10):
        self.layer1 = torch.nn.Conv2D(3, 5, 3)
        ln_shape = Network.calc_activation_shape(idim, 3) # <--- Calculate the shape of output of Convolution
        self.norm1 = torch.nn.LayerNorm([5, *ln_shape]) # <--- Normalize activations over C, H, and W (see fig.above)
        self.layer2 = torch.nn.Conv2D(5, 10, 3)
        ln_shape = Network.calc_activation_shape(ln_shape, 3)
        self.norm2 = torch.nn.LayerNorm([10, *ln_shape])
        self.layer3 = torch.nn.Dense(num_classes)


    def __call__(self, inputs):
        x = F.relu(self.norm1(self.layer1(input)))
        x = F.relu(self.norm2(self.layer2(x)))
        x = F.sigmoid(self.layer3(x))
        return x

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns

# 파이토치 텐서 생성
data = torch.randn(100, 10)  # 100개의 샘플과 10개의 피처

# 정규화 수행
mean = data.mean(dim=0)
std = data.std(dim=0)
data_normalized = (data - mean) / std

# 텐서를 NumPy 배열로 변환 (시각화를 위해)
data_np = data.numpy()
data_normalized_np = data_normalized.numpy()

# 기술통계 출력
print("Mean before normalization:", mean)
print("Standard deviation before normalization:", std)
print("Mean after normalization:", data_normalized.mean(dim=0))
print("Standard deviation after normalization:", data_normalized.std(dim=0))

# 첫 번째 피처에 대한 히스토그램 시각화
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(data_np[:, 0], kde=True, color='blue')
plt.title('Histogram before Normalization')
plt.xlabel('Feature Values')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
sns.histplot(data_normalized_np[:, 0], kde=True, color='red')
plt.title('Histogram after Normalization')
plt.xlabel('Feature Values')
plt.ylabel('Frequency')

plt.tight_layout()
# plt.show()

plt.title('Sample Plot')
plt.savefig('/workspace/0_practice/torch/features/image.png')  # 이미지 파일로 저장
plt.close()

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 생성
data = torch.randn(4, 3, 32, 32)  # 정규 분포에서 무작위 데이터 생성

# 정규화 층 초기화
batch_norm = nn.BatchNorm2d(3)  # 채널 수에 맞춰서
layer_norm = nn.LayerNorm([3, 32, 32])  # 정규화할 차원을 명시
group_norm = nn.GroupNorm(3, 3)  # 그룹 수와 채널 수

# 정규화 적용
data_bn = batch_norm(data.clone())  # 데이터를 복제하여 정규화
data_ln = layer_norm(data.clone())
data_gn = group_norm(data.clone())

# 시각화 함수
def plot_histogram(data, title):
    data = data.numpy().flatten()  # 히스토그램을 위해 데이터를 1차원 배열로 변환
    sns.histplot(data, kde=True, bins=30, color='blue')
    plt.title(title)
    plt.xlabel('Values')
    plt.ylabel('Frequency')
    plt.show()

# 원본 데이터 시각화
plt.figure(figsize=(12, 6))
plt.subplot(2, 2, 1)
plot_histogram(data, 'Original Data Histogram')

# 배치 정규화 데이터 시각화
plt.subplot(2, 2, 2)
plot_histogram(data_bn, 'BatchNorm Data Histogram')

# 레이어 정규화 데이터 시각화
plt.subplot(2, 2, 3)
plot_histogram(data_ln, 'LayerNorm Data Histogram')

# 그룹 정규화 데이터 시각화
plt.subplot(2, 2, 4)
plot_histogram(data_gn, 'GroupNorm Data Histogram')

### Instance Norm

In [ ]:
class InstanceNorm(Module):
    """
    https://nn.labml.ai/normalization/instance_norm/index.html
    """
    def __init__(self, channels, eps=1e-5, affine=True):
        super().__init__()
        self.channels = channels
        self.eps = eps
        self.affine = affine
        self.scale = nn.Parameter(torch.ones(channels))
        self.shift = nn.Parameter(torch.zeros(channels))

    def forward(slef, x:torch.Tensor):
        x_shape = x.shape
        batch_size = x_shape[0]

        assert self.channels = x.shape[1]
        x = x.view(batch_size, self.channels, -1)

        mean = x.mean(dim=-1, keepdim=True) # E[x]

        mean_x2 = (x**2).mean(dim=-1, keepdim=True) # E[x^2]

        var = mean_x2 - mean**2 # E[x^2] - E[x]^2

        x_norm = (x - mean) / torch.sqrt(var + self.eps)

        if self.affine:
        x_norm = self.scale.view(1, -1, 1) * x_norm + self.shift.view(1, -1, 1)

        return x_norm.view(x_shape)